In [3]:
import pickle
import random

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import numpy as np

from recipedataset import RecipeDataset, RECIPE_DATASET_FILENAME, NUM_GRAIN_SLOTS, NUM_ADJUNCT_SLOTS, NUM_HOP_SLOTS, NUM_MISC_SLOTS, NUM_MICROORGANISM_SLOTS, NUM_FERMENT_STAGE_SLOTS, NUM_MASH_STEPS

RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
torch.backends.cudnn.deterministic = True

def layer_init_ortho(layer, std=np.sqrt(2)):
  nn.init.orthogonal_(layer.weight, std)
  nn.init.constant_(layer.bias, 0.0)
  return layer

def layer_init_xavier(layer, gain):
  nn.init.xavier_normal_(layer.weight, gain)
  nn.init.constant_(layer.bias, 0.0)
  return layer

def reparameterize(mu, logvar):
  std = torch.exp(0.5 * logvar)
  eps = torch.randn_like(std)
  return eps * std + mu

In [4]:
BATCH_SIZE = 256

# Load the dataset and create a dataloader for it
with open("../" + RECIPE_DATASET_FILENAME, 'rb') as f:
  dataset = pickle.load(f)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)

In [6]:
# Determine the number of inputs to the network...

GRAIN_TYPE_EMBED_SIZE         = 32
ADJUNCT_TYPE_EMBED_SIZE       = 32
HOP_TYPE_EMBED_SIZE           = 96
MISC_TYPE_EMBED_SIZE          = 64
MICROORGANISM_TYPE_EMBED_SIZE = 64

# (boil_time + mash_ph + sparge_temp)
num_simple_inputs = 3 
# Mash steps (step_type_index_size + step_time + step_temp) * (number of slots) - ordering assumed [0: step 1, 1: step 2, etc.]
num_mash_step_inputs = NUM_MASH_STEPS*(len(dataset.mash_step_idx_to_name) + 2)
# Fermentation stages (step_time + step_temp) * (number of stages) - ordering assumed [0: primary, 1: secondary]
num_ferment_step_inputs = NUM_FERMENT_STAGE_SLOTS*(2)
# Grain/Malt bill slots (grain_type_embed_size + amount) * (number of slots) - no ordering
num_grain_slot_inputs = NUM_GRAIN_SLOTS*(GRAIN_TYPE_EMBED_SIZE + 1)
# Adjunct slots (adjunct_type_embed_size + amount) * (number of slots) - no ordering
num_adjunct_slot_inputs = NUM_ADJUNCT_SLOTS*(ADJUNCT_TYPE_EMBED_SIZE + 1)
# Hop slots (hop_type_embed_size + stage_type_index_size + time + concentration) * (number of slots) - no ordering
num_hop_slot_inputs = NUM_HOP_SLOTS*(HOP_TYPE_EMBED_SIZE + len(dataset.hop_stage_idx_to_name) + 2)
# Misc. slots (misc_type_embed_size + stage_type_index_size + time + amounts) * (number of slots) - no ordering
num_misc_slot_inputs = NUM_MISC_SLOTS*(MISC_TYPE_EMBED_SIZE + len(dataset.misc_stage_idx_to_name) + 2)
# Microorganism slots (mo_type_embed_size + stage_type_index_size) * (number of slots) - no ordering
num_mo_slot_inputs = NUM_MICROORGANISM_SLOTS*(MICROORGANISM_TYPE_EMBED_SIZE + len(dataset.mo_stage_idx_to_name))

num_inputs = num_simple_inputs + num_mash_step_inputs + num_ferment_step_inputs + num_grain_slot_inputs + \
             num_adjunct_slot_inputs + num_hop_slot_inputs + num_misc_slot_inputs + num_mo_slot_inputs
print(num_inputs)

5899


In [ ]:



class RecipeVAE(nn.Module):
  
  def __init__(
    self, 
    num_inputs:int, num_hidden_layers:int, hidden_size:int, z_size:int, 
    activation_fn=nn.ELU(), 
    gain=nn.init.calculate_gain('linear')
  ) -> None:
    
    super().__init__()
    assert num_inputs >= 1
    assert num_hidden_layers >= 1
    assert hidden_size >= 1
    assert z_size >= 1 and z_size < num_inputs
    
    #LEAKY_RELU_SLOPE = 0.1
    z_sizex2 = z_size*2

    self.encoder = nn.ModuleList()
    self.encoder.append(layer_init_xavier(nn.Linear(num_inputs, hidden_size), gain))
    self.encoder.append(activation_fn)
    
    
    for _ in range(1, num_hidden_layers):
      self.encoder.append(layer_init_xavier(nn.Linear(hidden_size, hidden_size), gain))
      self.encoder.append(activation_fn)
      
    self.encoder.append(layer_init_xavier(nn.Linear(hidden_size, z_sizex2), gain))
    self.encoder.append(activation_fn(inplace=True))
    self.encoder.append(nn.BatchNorm1d(z_sizex2))

    self.decoder = nn.ModuleList([
      
    ])
    
    #enc1 = layer_init_xavier(nn.Linear(NUM_GRAIN_SLOTS, HIDDEN_SIZE), 'leaky_relu', LEAKY_RELU_SLOPE)
    #enc2 = layer_init_xavier(nn.Linear(HIDDEN_SIZE, 2*Z_SIZE), 'linear')
    #dec1 = layer_init_xavier(nn.Linear(Z_SIZE, HIDDEN_SIZE), 'leaky_relu', LEAKY_RELU_SLOPE)
    #dec2 = layer_init_xavier(nn.Linear(HIDDEN_SIZE, NUM_GRAIN_SLOTS), 'tanh')
    
    
    
  def forward(self, x):
    pass
  


  
  